In [300]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [301]:
diabetes=pd.read_csv("diabetess.csv")
diabetes

,erythema,scaling,definite borders,itching,koebner phenomenon,polygonal papules,follicular papules,oral mucosal involvement,knee and elbow involvement,scalp involvement,...,disappearance of the granular layer,vacuolisation and damage of basal layer,spongiosis,saw-tooth appearance of retes,follicular horn plug,perifollicular parakeratosis,inflammatory monoluclear inflitrate,band-like infiltrate,Age,class
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55.0,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8.0,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26.0,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40.0,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2,1,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,2,0,25.0,4
362,3,2,1,0,1,0,0,0,0,0,...,1,0,1,0,0,0,2,0,36.0,4
363,3,2,2,2,3,2,0,2,0,0,...,0,3,0,3,0,0,2,3,28.0,3
364,2,1,3,1,2,3,0,2,0,0,...,0,2,0,1,0,0,2,3,50.0,3


In [302]:
diabetes["Age"].isnull().sum()

8

In [303]:
diabetes["Age"]=diabetes["Age"].fillna(diabetes["Age"].mean())

In [304]:
diabetes["Age"]

0      55.0
1       8.0
2      26.0
3      40.0
4      45.0
       ... 
361    25.0
362    36.0
363    28.0
364    50.0
365    35.0
Name: Age, Length: 366, dtype: float64

In [305]:
diabetes.dtypes

erythema                                      int64
scaling                                       int64
definite borders                              int64
itching                                       int64
koebner phenomenon                            int64
polygonal papules                             int64
follicular papules                            int64
oral mucosal involvement                      int64
knee and elbow involvement                    int64
scalp involvement                             int64
family history                                int64
melanin incontinence                          int64
eosinophils in the infiltrate                 int64
PNL infiltrate                                int64
fibrosis of the papillary dermis              int64
exocytosis                                    int64
acanthosis                                    int64
hyperkeratosis                                int64
parakeratosis                                 int64
clubbing of 

In [306]:
X=diabetes.drop(columns="class")
y=diabetes["class"]

In [307]:
from sklearn.decomposition import PCA,FactorAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Model without Factor Analysis

In [308]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=15)

In [309]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [310]:
classifier=RandomForestClassifier(random_state=10)
model_fit=classifier.fit(X_train,y_train)

In [311]:
y_test_pred=model_fit.predict(X_test)
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        23
           2       0.91      0.83      0.87        12
           3       1.00      1.00      1.00        15
           4       0.82      0.90      0.86        10
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00         4

    accuracy                           0.96        74
   macro avg       0.95      0.96      0.95        74
weighted avg       0.96      0.96      0.96        74



### Model with Factor Analysis

In [312]:
correlation_matrix=X_train.corr(method='kendall').abs()
unstacked_corr=correlation_matrix.unstack()
ordered=unstacked_corr.sort_values(ascending=False)

In [313]:
count=0
for i in range(len(ordered.values)):
    if ordered.values[i] < 1.0 and ordered.values[i] >= 0.5:
        count = count + 1
print(count)

92


In [314]:
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [321]:
factor=FactorAnalysis(n_components=20)
factor_fit=factor.fit(X_train_scaled)

In [322]:
X_train_factor=factor_fit.transform(X_train_scaled)
X_test_factor=factor_fit.transform(X_test_scaled)

In [323]:
X_train_factor

array([[-0.7103687 , -0.97695437,  0.92647591, ..., -0.01065604,
         0.13858817, -0.46004667],
       [-0.73541644, -1.17312711,  1.24922004, ...,  0.34515539,
         0.61776578,  0.33476934],
       [-0.55027335,  3.95977149,  2.56555655, ...,  1.10081819,
         0.70312076, -0.42945089],
       ...,
       [ 2.53985161, -0.18772305,  0.47230517, ..., -1.81862315,
        -0.99813315, -0.16512624],
       [-0.42159496,  0.22232192, -0.85166056, ..., -0.33362047,
        -0.59276471, -0.75384967],
       [ 2.08489061, -0.21264172,  0.35950152, ..., -0.27042411,
         0.82820332,  0.2702102 ]])

In [324]:
X_train_factor.shape

(292, 20)

In [325]:
classifier=RandomForestClassifier(random_state=18)
model_fit=classifier.fit(X_train_factor,y_train)

In [326]:
y_test_pred=model_fit.predict(X_test_factor)
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        23
           2       0.86      1.00      0.92        12
           3       1.00      1.00      1.00        15
           4       1.00      0.80      0.89        10
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00         4

    accuracy                           0.97        74
   macro avg       0.98      0.97      0.97        74
weighted avg       0.98      0.97      0.97        74

